# Import packages et données

In [1]:
# Chargement package
import pandas as pd
import numpy as np


In [2]:
# Connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import du fichier via ggdrive
file_path = "/content/drive/MyDrive/M2_SISE/Python_M2/train.csv" # Lien perso ggdrive
df = pd.read_csv(file_path, sep = ';')

# Préparation des données

## Recodage variables

In [4]:
# Liste des colonnes à transtyper en numeric
var_tofloat = ['int_corr','income','pf_o_att',	'pf_o_sin',	'pf_o_int',	'pf_o_fun',	'pf_o_amb',	'pf_o_sha', 'attr1_1', 'sinc1_1',	'intel1_1',	'fun1_1',	'amb1_1',	'shar1_1']

for i in var_tofloat:
  df[i] = df[i].str.replace(",","")
  df[i] = pd.to_numeric(df[i], downcast="float")



In [5]:
# Liste des colonnes à transtyper en object
var_toobject = ['gender','idg', 'condtn',	'wave', 'position',	'positin1','match','samerace','race_o','dec_o','field_cd', 'race', 'goal','date', 'go_out', 'career_c']

for i in var_toobject:
  df[i] = df[i].astype("object")


In [6]:
df.shape

(6804, 70)

In [7]:
# Remove field_cd career_c, codage de field et career
df = df.drop(columns=['field_cd', 'career_c'])
df.shape

(6804, 68)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6804 entries, 0 to 6803
Data columns (total 68 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   iid_pid   6804 non-null   int64  
 1   iid       6804 non-null   int64  
 2   id        6804 non-null   int64  
 3   gender    6804 non-null   object 
 4   idg       6804 non-null   object 
 5   condtn    6804 non-null   object 
 6   wave      6804 non-null   object 
 7   round     6804 non-null   int64  
 8   position  6804 non-null   object 
 9   positin1  4958 non-null   object 
 10  order     6804 non-null   int64  
 11  partner   6804 non-null   int64  
 12  pid       6794 non-null   float64
 13  match     6804 non-null   object 
 14  int_corr  6646 non-null   float32
 15  samerace  6804 non-null   object 
 16  age_o     6722 non-null   float64
 17  race_o    6731 non-null   object 
 18  pf_o_att  6715 non-null   float32
 19  pf_o_sin  6715 non-null   float32
 20  pf_o_int  6715 non-null   floa

## NaN Values

### Numeric variables

In [9]:
# Pourcentage de valeurs NaN dans chacune des colonnes (> 0)
na_num = df.select_dtypes(exclude='object').isna().sum()/len(df.select_dtypes(exclude='object'))
na_num = na_num[na_num > 0].sort_values(ascending=False)

# 10 premières colonnes numeric avec le plus de NaN
na_num.head(10)

expnum      0.735450
income      0.477366
shar_o      0.135655
amb_o       0.085391
fun_o       0.040711
intel_o     0.033657
sinc_o      0.030423
int_corr    0.023222
attr_o      0.020723
shar1_1     0.016608
dtype: float64

In [10]:
# Création d'un dataset avec les variables quanti/numeric
df_quanti = df.select_dtypes(exclude='object')
print("Format : " )
print(df_quanti.shape)
print("Nombre de NaN : ")
print(df_quanti.isna().sum().sum())


Format : 
(6804, 50)
Nombre de NaN : 
13952


In [11]:
df_quanti.columns

Index(['iid_pid', 'iid', 'id', 'round', 'order', 'partner', 'pid', 'int_corr',
       'age_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb',
       'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
       'age', 'imprace', 'imprelig', 'income', 'sports', 'tvsports',
       'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing',
       'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping',
       'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1', 'intel1_1',
       'fun1_1', 'amb1_1', 'shar1_1'],
      dtype='object')

In [12]:
# Remplacer les NaN avec KNNImputer()
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="distance")

df_quanti = pd.DataFrame(imputer.fit_transform(df_quanti), columns = df_quanti.columns)

In [13]:
# Nombre de NaN
print("Nombre de NaN : ")
print(df_quanti.isna().sum().sum())

Nombre de NaN : 
0


KNN calcule la valeur moyenne des 2 plus proches voisins, les valeurs manquantes sont donc remplacés par un nombre réel au lieu d'un entier. A voir s'il y a besoin de les arrondir pour certaines variables 

### Object variables

In [14]:
# Pourcentage de valeurs NaN dans chacune des colonnes (> 0)
na_obj = df.select_dtypes(include='object').isna().sum()/len(df.select_dtypes(include='object'))
na_obj = na_obj[na_obj > 0].sort_values(ascending=False)

# Colonnes object avec le plus de NaN
na_obj

positin1    0.271311
zipcode     0.131687
date        0.014256
career      0.013081
from        0.011611
goal        0.011611
go_out      0.011611
race_o      0.010729
field       0.009259
race        0.009259
dtype: float64

In [15]:
df_quali = df.select_dtypes(include='object')
print("Format : " )
print(df_quali.shape)
print("Nombre de NaN : ")
print(df_quali.isna().sum().sum())

Format : 
(6804, 18)
Nombre de NaN : 
3364


In [16]:
# Remplace les valeurs manquantes par Missing value 
df_quali.fillna("Missing value", inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [17]:
print("Format : " )
print(df_quali.shape)
print("Nombre de NaN : ")
print(df_quali.isna().sum().sum())

Format : 
(6804, 18)
Nombre de NaN : 
0


### Fusion des datasets 

In [18]:
# Combinaison des datasets df_quanti et df_quali 
df_clean = pd.concat([df_quanti, df_quali.reindex(df_quanti.index)], axis=1)
df_clean.shape

(6804, 68)

In [19]:
df_clean.isna().sum().sum()

0

### Export fichier en csv

In [20]:
from pathlib import Path  
filepath = Path('/content/drive/MyDrive/M2_SISE/Python_M2/df_dating.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_clean.to_csv(filepath)  